In [1]:
import pandas as pd
from ast import literal_eval
literals = {
  'mfcc':  literal_eval,
  'chroma': literal_eval
}
df = pd.read_csv('rnn_features.csv', converters=literals)

import numpy as np
df = df[['label', 'mfcc', 'chroma']]

In [2]:
df.groupby('label').size()

label
Romania            6
Southern-Italy    29
Thessaly          20
Thrace            31
Turkey             4
dtype: int64

In [3]:
df['mfcc'] = df['mfcc'].apply(lambda x: np.array(x))
df['chroma'] = df['chroma'].apply(lambda x: np.array(x))

In [4]:
dataset = df.values
labels = dataset[:, 0]
dataset = dataset[:, 1:]

In [5]:
# Create an empty list to store the concatenated arrays
concatenated_data = []

# Iterate over each pair of arrays in the data
for array1, array2 in dataset:
    # Concatenate the arrays along the first dimension (axis 0)
    concatenated_array = np.concatenate((array1, array2), axis=0)
    # Append the concatenated array to the list
    concatenated_data.append(concatenated_array.T)

# Convert the list of concatenated arrays to a numpy array
final_data = np.array(concatenated_data)

dataset = final_data

In [6]:
dataset.shape

(90, 100, 25)

In [ ]:
encoding = {
    'Romania': 0,          
    'Southern-Italy': 1,  
    'Thessaly': 2,         
    'Thrace': 3,           
    'Turkey': 4
}

In [ ]:
labels2 = [0] * len(labels)
for i,label in enumerate(labels):
    labels2[i] = encoding[label]

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sound_utils import *
from data_utils import *
from model import *
from model_utils import ModelUtils

num_classes = 5 
dataset = AudioDataset(dataset, labels2)

# Define the sizes for the splits
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

input_shape = 25

In [ ]:
model_experiment = ModelUtils(train_loader,val_loader,test_loader,input_shape,num_classes)

epochs = 1
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

params = {
            'learning_rate': 0.0001,
            'num_layers': 2,
            'skip_connections': False,
            'dropout_probability': 0.5,
            'hidden_layers': 2,
            'gradient_clipping': False,
            'cell_type': 'rnn',
            'hidden_size': 128,
            'bidirectional': False
            }

model_experiment.train_evaluate(params)

In [ ]:
model_experiment.test_model()